In [35]:
import pandas as pd
import psycopg2
from psycopg2 import extras
from config import settings
import logging
import warnings

# Initialize your logger
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def create_connection():
    try:
        conn = psycopg2.connect(
            host=settings.predict_db_host,
            port=settings.predict_db_port,
            dbname=settings.predict_db_name,
            user=settings.predict_db_username,
            password=settings.predict_db_password
        )
        return conn
    except psycopg2.Error as e:
        logger.error(f"Error connecting to PostgreSQL database: {e}")
        return None

def insert_into_batch_info(df):
    conn = create_connection()
    if conn is not None:
        with conn.cursor() as cursor:
            try:
                # Generate SQL for batch insertion
                columns = df.columns.tolist()
                values = [tuple(x) for x in df.to_numpy()]
                insert_query = f"INSERT INTO prediction.batch_info({','.join(columns)}) VALUES %s"
                extras.execute_values(cursor, insert_query, values)
                conn.commit()
                logger.info("Data inserted into batch_info successfully.")
            except psycopg2.Error as e:
                conn.rollback()
                logger.error(f"Failed to insert into batch_info: {e}")
            finally:
                cursor.close()
        conn.close()

def insert_into_result(df):
    conn = create_connection()
    if conn is not None:
        with conn.cursor() as cursor:
            try:
                columns = df.columns.tolist()
                values = [tuple(x) for x in df.to_numpy()]
                insert_query = f"INSERT INTO prediction.result({','.join(columns)}) VALUES %s"
                extras.execute_values(cursor, insert_query, values)
                conn.commit()
                logger.info("Data inserted into result successfully.")
            except psycopg2.Error as e:
                conn.rollback()
                logger.error(f"Failed to insert into result: {e}")
            finally:
                cursor.close()
        conn.close()

def insert_into_interface(df):
    conn = create_connection()
    if conn is not None:
        with conn.cursor() as cursor:
            try:
                columns = df.columns.tolist()
                values = [tuple(x) for x in df.to_numpy()]
                insert_query = f"INSERT INTO prediction.interface({','.join(columns)}) VALUES %s"
                extras.execute_values(cursor, insert_query, values)
                conn.commit()
                logger.info("Data inserted into interface successfully.")
            except psycopg2.Error as e:
                conn.rollback()
                logger.error(f"Failed to insert into interface: {e}")
            finally:
                cursor.close()
        conn.close()

def fetch_all_data(table_name):
    conn = create_connection()  # Assuming create_connection() is already defined
    if conn is not None:
        try:
            sql_query = f"SELECT * FROM {table_name}"
            data = pd.read_sql(sql_query, conn)
            logger.info(f"Data fetched from {table_name} successfully.")
            return data
        except psycopg2.Error as e:
            logger.error(f"Failed to fetch data from {table_name}: {e}")
            return None
        finally:
            conn.close()
    else:
        logger.error("Connection to database failed.")
        return None

def get_all_batch_info():
    return fetch_all_data("prediction.batch_info")

def get_all_results():
    return fetch_all_data("prediction.result")

def get_all_interfaces():
    return fetch_all_data("prediction.interface")

In [41]:
# data = pd.read_csv('batch_info.csv').drop(columns=('Unnamed: 0'))
# data
# # batch_data = ('Slot2', 'Material1', 'Batch123', 'A001', 'operation1', 25.5)
# insert_into_batch_info(data)
get_all_batch_info()

/tmp/ipykernel_257588/891335765.py:86: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(sql_query, conn)
INFO:__main__:Data fetched from prediction.batch_info successfully.


,id,inslot,material,batch,plant,operationno,sampleweight,created_at,updated_at
0,1,Slot2,Material1,Batch123,A001,operation1,25.5,2024-05-10 06:52:57.655499+00:00,2024-05-10 06:52:57.655499+00:00
1,2,10005546570,V071100002,I240220001,A003,10,99.9,2024-05-10 09:11:23.627833+00:00,2024-05-10 09:11:23.627833+00:00
2,3,10005546570,V071100002,I240220001,A003,0010,99.9,2024-05-10 09:35:37.485160+00:00,2024-05-10 09:35:37.485160+00:00


In [40]:
# data = pd.read_csv('result.csv').drop(columns=('Unnamed: 0'))
# data
# # batch_data = ('Slot2', 'Material1', 'Batch123', 'A001', 'operation1', 25.5)
# insert_into_result(data)
get_all_results()

/tmp/ipykernel_257588/891335765.py:86: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(sql_query, conn)
INFO:__main__:Data fetched from prediction.result successfully.


,id,inslot,material,batch,plant,operationno,sampleweight,c1c,c2c,c3c,...,c6w,c7w,c36w,c37w,c38w,c39w,c40w,total_weight,created_at,updated_at
0,1,Slot2,Material1,Batch123,A001,operation1,25.5,10,20,30,...,6.600000,7.7,8.800000,9.900000,10.100000,11.200000,12.300000,123.45,2024-05-10 06:53:13.535675+00:00,2024-05-10 06:53:13.535675+00:00
1,2,10005546570,V071100002,I240220001,A003,10,99.9,245,8,0,...,0.000000,0.0,3.939718,10.974930,0.562817,0.281408,0.281408,99.90,2024-05-10 09:27:57.037449+00:00,2024-05-10 09:27:57.037449+00:00
2,3,10005546570,V071100002,I240220001,A003,0010,99.9,240,6,0,...,0.561236,0.0,2.806180,12.066573,0.561236,0.561236,0.280618,99.90,2024-05-10 09:35:37.491602+00:00,2024-05-10 09:35:37.491602+00:00
3,4,10005546570,V071100002,I240220001,A003,0010,99.9,235,7,0,...,0.000000,0.0,3.095493,14.633239,1.407042,0.281408,0.281408,99.90,2024-05-10 09:52:25.308344+00:00,2024-05-10 09:52:25.308344+00:00
4,5,10005546570,V071100002,I240220001,A003,0010,99.9,233,6,0,...,0.000000,0.0,4.209270,14.592135,1.403090,0.280618,0.000000,99.90,2024-05-10 09:53:14.988289+00:00,2024-05-10 09:53:14.988289+00:00
5,6,10005546570,V071100002,I240220001,A003,0010,99.9,232,5,0,...,0.000000,0.0,2.790000,16.740000,0.840000,0.000000,0.280000,99.90,2024-05-10 09:54:32.699149+00:00,2024-05-10 09:54:32.699149+00:00


In [42]:
# data = pd.read_csv('interface.csv').drop(columns=('Unnamed: 0'))
# data
# # batch_data = ('Slot2', 'Material1', 'Batch123', 'A001', 'operation1', 25.5)
# insert_into_interface(data)
get_all_interfaces()

/tmp/ipykernel_257588/891335765.py:86: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(sql_query, conn)
INFO:__main__:Data fetched from prediction.interface successfully.


,id,inslot,material,batch,plant,operationno,sampleweight,phys003,phys004,phys005,phys006,phys007,phys008,phys009,created_at,updated_at
0,1,Slot2,Material1,Batch123,A001,operation1,25.5,90.500000,10.200000,11.300000,12.4,0.000000,13.5,14.600000,2024-05-10 06:55:54.495618+00:00,2024-05-10 06:55:54.495618+00:00
1,2,10005546570,V071100002,I240220001,A003,10,99.9,15.774648,0.000000,4.507042,0.0,0.563380,0.0,10.985915,2024-05-10 09:33:53.621813+00:00,2024-05-10 09:33:53.621813+00:00
2,3,10005546570,V071100002,I240220001,A003,0010,99.9,16.853933,0.561798,2.808989,0.0,1.123596,0.0,11.235955,2024-05-10 09:35:37.499928+00:00,2024-05-10 09:35:37.499928+00:00
3,4,10005546570,V071100002,I240220001,A003,0010,99.9,19.154930,0.000000,3.098592,0.0,1.690141,0.0,9.859155,2024-05-10 09:52:25.314619+00:00,2024-05-10 09:52:25.314619+00:00
4,5,10005546570,V071100002,I240220001,A003,0010,99.9,18.539326,0.000000,4.213483,0.0,1.685393,0.0,10.112360,2024-05-10 09:53:14.993576+00:00,2024-05-10 09:53:14.993576+00:00
5,6,10005546570,V071100002,I240220001,A003,0010,99.9,20.950000,0.000000,2.790000,0.0,0.840000,0.0,10.610000,2024-05-10 09:54:32.704140+00:00,2024-05-10 09:54:32.704140+00:00
